<span style="color:blueviolet">**KODLISTOR_STOR: *k_id - kodlista - kod - svarstext***   
kanske går att hämta från min lista med dictionairies? 
  
  
<span style="color:blueviolet">**FREKVENSTABELL: *v_id - variabel - enkät - kod - frekvens***   
blir lite klurigare... jag måste göra något åt att det står "w+n" i enkätkolumnen. För att hitta rätt kod och frekvens borde jag kunna söka igenom den "högra delen" av filen, alltså där frekvenstabellerna ligger, och välja ut de celler som faktiskt har värden. FÖrmodligen står det NULL i de tomma cellerna efter att jag läst in excelfilen till en dataframe.  
  
    
    
<span style="color:blueviolet">**VARIABLER: *variabel - enkät - beskrivning - itemnr - enkättext - kodlista***   
borde inte vara så svår. Dela på på enkätkolumnen så att w+n separeras. Sedan måste item# hämtas från den högra delen av tabellen... det borde inte vara så klurigt.

# Innehållsförteckning  
1. [Introduktion](#1)  
2. [Kolumner](#2)  
    2.1 [Enkät](#2.1)  
    2.2 [Itemnr](#2.2)  
    2.3 [Frekvens](#2.3)
3. [Tabeller](#3)  
    3.1 [Variabler](#3.1)  
    3.2 [Frekvenstabell](#3.2)  
    3.3 [Enkättexter](#3.3)  
    3.4 [Kodlistor](#3.4)  
    3.5 [Svarstexter](#3.5)  
    3.6 [Kodlistor_stor](#3.6)

***

# <a id = "1">1. Introduktion</a>  
Den metadata som redan är dokumenterad enligt Vetenskapsrådets riktlinjer ligger i excelfilen *Metadata*. Några av tabellerna i relationsdatabsen innehåller kolumner som inte redan finns i excelfilen *Metadata*. Dessa kolumner skapas i avsnittet [Kolumner](#2). Därefter kommer följande tabeller att skapas:  

|Variabler|Frekvenstabell|Enkättexter|Kodlistor|Svarstexter|Kodlistor_stor|  
|:---:|:---:|---|---|---|---|
|Variabel|V_id|Enkättext|Kodlista|Svarstext|K_id|  
|Enkät|Variabel|Källa|From|From|Kodlista|  
|Beskrivning|Enkät|-|Tom|Tom|Kod|  
|Itemnr|Kod|-|-|-|Svarstext|  
|Enkättext|Frekvens|-|-|-|-|  
|Kodlista|-|-|-|-|-|  
  
  
Relationsdatabasen kommmer att innehålla ytterligare fyra tabeller. Dessa är **Begrepp**, **Begrepp_stor**, **Tidsserie** samt **Tidsserie_stor**. För tillfället finns ingen data till dessa tabeller i *Metadata*, därför kommer de inte skapas här. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('Metadata.xlsx')

För att ta fram **Kodlista_stor** används `missing`-värdet i text-kolumnen. Eftersom `missing` inte är lika med `Missing` blir det problem. För att undvika detta ersätts `missing` med `Missing`.

In [3]:
df.replace('missing', 'Missing', inplace = True)

In [4]:
df.columns

Index(['Namn', 'Tidsserie', 'Beskrivning', 'Enkät', 'Dubbelkodning',
       'Enkättext', 'From', 'Tom', 'Kodlista', 'From.1', 'Tom.1', 'Kod',
       'Text', 'From.2', 'Tom.2', 'Koncept 1', 'Koncept 2', 'Instrument 1',
       'Instrument 2', 'Källa', 'SLOSHW06', 'frek', 'SLOSHW08', 'frek.1',
       'SLOSHW10', 'frek.2', 'SLOSHW12', 'frek.3', 'SLOSHW14', 'frek.4',
       'SLOSHW16', 'frek.5', 'SLOSHW18', 'frek.6', 'SLOSHNW06', 'frek.7',
       'SLOSHNW08', 'frek.8', 'SLOSHNW10', 'frek.9', 'SLOSHNW12', 'frek.10',
       'SLOSHNW14', 'frek.11', 'SLOSHNW16', 'frek.12', 'SLOSHNW18', 'frek.13'],
      dtype='object')

*** 
# <a id = "2">2. Kolumner</a>  
Kolumnerna som måste skapas från *Metadata* är Enkät, Itemnr och Frekvens.  

***  
## <a id = "2.1">2.1 Enkät</a>  
Den här kolumnen ska innehålla en `str` som säger vilken år (eller våg) variabeln förekommer i. Samma `str` ska säga huruvida variabeln finns för den arbetande- eller den icke-arbetande delen av SLOSH. Ett exempel på ett ett värde i kolumnen är "SLOSHW08". Här står "W" för "working" ("non-working" förkortas "NW") och "08" för den andra SLOSH-vågen (som ägde rum 2008). Värdena i den här kolumnen är namn på (redan existerande) kolumner i *Metadata*. Börja med att samla namnen på working-kolumner respektive nonworking-kolumner i två listor. Listorna kallas "WORKING" respektive "NONWORKING".

In [5]:
WORKING = ['SLOSHW06', 'SLOSHW08', 'SLOSHW10', 'SLOSHW12', 'SLOSHW14', 'SLOSHW16', 'SLOSHW18']
NONWORKING = ['SLOSHNW06', 'SLOSHNW08', 'SLOSHNW10', 'SLOSHNW12', 'SLOSHNW14', 'SLOSHNW16', 'SLOSHNW18']

Kolumnerna som står för vågor i SLOSH (alltså t.ex. SLOSHW06) innehåller `NaN` ifall en given variabel inte finns med i vågen. Om variabeln istället *finns* kommer vågkolumnen innehålla en `str`. De kolumner som heter "SLOSHW" står för enkäter för arbetande i SLOSH. Ifall kolumnen börjar med "SLOSHNW" står den för en enkät för icke-arbetande. Varje variabel kan förekomma i högst två kolumner (en SLOSHW och en SLOSHNW). Ifall en given variabel förekommer i *endast en* kolumn ska det i Enkät-kolumnen stå t.ex. `SLOSHW06`.

In [6]:
for i,j in zip(WORKING, NONWORKING):
    df.loc[(df[i].isna() == False) & (df[j].isna() == True), ['Enkät']] = i
    df.loc[(df[j].isna() == False) & (df[i].isna() == True), ['Enkät']] = j

In [7]:
df['Enkät'].value_counts()

SLOSHW06       1210
SLOSHW08        889
w+n             748
SLOSHW10        669
SLOSHW12        579
SLOSHW14        567
SLOSHW16        517
SLOSHW18        426
SLOSHNW08       307
working         181
SLOSHNW12       160
SLOSHNW14       130
SLOSHNW16       106
SLOSHNW18        75
SLOSHNW10        41
non-working      34
SLOSHNW06        24
Name: Enkät, dtype: int64

Frekvenstabellen visar att det finns tre värden i Enkät-kolumnen för vilka tillhörande variabler förekommer i två vågkolumner. Detta är oväntat. Alla variabler som förekommer i två vågor ska egentligen ha Enkät-värdet `w+n`. Alltså har vi dokumenterat fel på något sätt. Tillsvidare får dessa variabler Enkät-värdet `oklar`.

In [8]:
df.loc[(df['Enkät'] == 'w+n') | (df['Enkät'] == 'working') | (df['Enkät'] == 'non-working'), ['Enkät']] = 'oklar'

  ***  
## <a id = "2.2">2.2 Itemnr</a>  
Nu passar Enkät-kolumnen in i relationsdatabasen (med undantag för de variabler som är "oklara"). Kolumnen Itemnr ska innehålla `str` som för närvarande ligger utspridda i vågkolumnerna i *Metadata*. Ett värde i kolumnen kan t.ex. vara `A26_b`. Här är ett förenklat schema som visar relationen mellan Itemnr-kolumnen och vågkolumnerna i *Metadata*:

In [31]:
pd.DataFrame([['A25', 'NaN', 'Nan', 'A25'], ['NaN', 'B23', 'Nan', 'B23'], ['NaN', 'NaN', 'C13', 'C13']], 
             columns = ['SLOSHW06', 'SLOSHNW10', 'SLOSHW16', 'Itemnr'])

,SLOSHW06,SLOSHNW10,SLOSHW16,Itemnr
0,A25,NaN,Nan,A25
1,NaN,B23,Nan,B23
2,NaN,NaN,C13,C13


Första skapas en ny kolumn som får namnet "Itemnr", kolumnen fylls med `NaN`.

In [26]:
df['Itemnr'] = np.nan

Proceduren för att välja ut rätt itemnummer illustreras i testet nedan.  
<span style="color:red">**TEST**

In [10]:
d = pd.DataFrame([[1,2,3], [4,5,6], [1,8,9]], columns = ['A', 'B', 'C'])
d['test'] = 0

In [11]:
d

,A,B,C,test
0,1,2,3,0
1,4,5,6,0
2,1,8,9,0


In [12]:
d.loc[d['A'] == 1, ['test']] = d['B']
d

,A,B,C,test
0,1,2,3,2
1,4,5,6,0
2,1,8,9,8


<span style="color:red">**TEST SLUT**  
***

Loopen nedan går igenom vågkolumnerna i *Metadata*. Om en variabel har fått värdet `SLOSHW06` i enkätkolumnen kommer värdet för samma variabel i vågkolumnen SLOSHW06 att bli variabelns värde i den nya kolumnen Itemnr.

In [13]:
for i,j in zip(WORKING, NONWORKING):
    df.loc[df['Enkät'] == i, ['Itemnr']] = df[i]
    df.loc[df['Enkät'] == j, ['Itemnr']] = df[j]

Varabler med enkätvärdet `oklar` ges Itemr `oklar`. 

In [ ]:
df.loc[df['Enkät'] == 'oklar', ['Itemnr']] = 'oklar'

Gör en `value_counts` för att se att någon frekvenstabell i *Metadata* hamnat på fel ställe... Kanske har den förskjutits någon kolumn eller så...

***  
## <a id = "2.3">2.3 Frekvens</a>  
Kolumnen med frekvenser görs på ungefär samma sätt som kolumnen med Itemnr. I *Metadata* ligger en frekvenskolumn intill varje vågkolumn. Exempelvis åtföljs kolumnen SLOSHW06 av en frekvenskolumn som rymmer frekvenser för de variabler som förekommer i SLOSHW06. De variabler som *inte* förekommer i SLOSHW06 har värdet `NaN`i samma frekvenskolumn.  
  
Skapa först en ny kolumn med namnet "Frekvens" och fyll den med `NaN`.


In [15]:
df['Frekvens'] = np.nan

Skapa två listor, en för arbetande och en för icke-arbetande, och fyll dessa med namnen på frekvenskolumnerna från *Metadata*.

In [16]:
WORKFREK = ['frek.1', 'frek.2', 'frek.3', 'frek.4', 'frek.5', 'frek.6', 'frek.7']
NONWORKFREK = ['frek.8', 'frek.9', 'frek.10', 'frek.11', 'frek.12', 'frek.13']

Loopen nedan går igenom frekvenskolumnerna i *Metadata*. De variabler som har frekvenser för högst en våg får detta värde i den nya frekvenskolumnen. De variabler som har frekvenser i två vågor, alltså de som i *Metadata* är `w+n` samt ej är dubbelkodade, får tillsvidare `NaN` i den nya frekvenskolumnen. Anledningen är att det i avsnittet [Enkät](#2.2) antyds att vissa variabler är feldokumenterade.

In [17]:
for i,j in zip(WORKFREK, NONWORKFREK):
    df.loc[(df[i].isna() == False) & (df[j].isna() == True), ['Frekvens']] = df[i]
    df.loc[(df[j].isna() == False) & (df[i].isna() == True), ['Frekvens']] = df[j]

***  
# <a id = "3">3. Tabeller</a>  
I det här avnittet kommer tabellerna (förutom de som rör tidsserre och begrepp, se [introduktionen](#1)) till relationsdatabasen att skapas. Först tabellen **Variabler** (som är störst). Därefter **Frekvenstabeller**, **Enkättexter**, **Kodlistor**, **Svarstexter** och till sist **Kodlistor_stor**. 

***  
## <a id = "3.1">3.1 Variabler</a>  

***  
## <a id = "3.2">3.2 Frekvenstabeller</a>  

***  
## <a id = "3.3">3.3 Enkättexter</a>  

***  
## <a id = "3.4">3.4 Kodlistor</a>  

***  
## <a id = "3.5">3.5 Svarstexter</a>  

***  
## <a id = "3.6">3.6 Kodlistor_stor</a>  
Den här tabellen ska innehålla kodlistornas namn tillsammans med deras kod och svarstext. Varje sådant sjok ska endast förekomma en gång. Börja med att skära ut kolumnerna Kodlista, Kod och Text ur *Metadata*.

In [18]:
d = df[['Kodlista', 'Kod', 'Text']]  
d.head(5)

I den här mindre tabellen förekommer vissa kodlistor flera gånger. För att identifiera dubletter behövs en slags räknare. Börja med att lägga till en ny kolumn fylld med `0`. Kalla kolumnen för K_id.

In [21]:
d['K_id'] = 0
d.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Index(['Kodlista', 'Kod', 'Text', 'kid'], dtype='object')

K_id ska vara unik för varje sjok. Loopen nedan ökar K_id med `1` varje gång den passerar ett sjok. Ett sjok identifieras genom att dess sista rad alltid har svarstexten `Missing`.

In [23]:
for i in range(1, len(d['K_id'])):
    if d.iloc[i-1, 2] == 'Missing':
        d.iloc[i: , 3] = d.iloc[i: , 3] + 1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Loopen nedan lägger varje kodlista i listan `behåll` endast en gång. Varje gång en kodlista läggs i `behåll` lägger loopen tillhörande K_id i listan `unikid`. Denna andra lista innehåller alltså alla unika förekomster av "sjok".

In [24]:
unikid = []
behåll = []
for i in range(len(d['K_id'])):
    if d.iloc[i, 0] not in behåll:
        unikid.append(d.iloc[i, 3])
        behåll.append(d.iloc[i, 0])
        

Tills sist skapas tabellen **Kodlistor_stor** genom att endast ha med de rader för vilka K_id återfinns i listan `unikid`.

In [25]:
Kodlistor_stor = d.loc[d['K_id'].isin(unikid)]